[Explanation of the Solution](https://www.kaggle.com/competitions/geology-forecast-challenge-open/discussion/582761)

**Code:**

["Generate dataset"](https://www.kaggle.com/code/act18l/generate-dataset)

["Single Model"](https://www.kaggle.com/code/act18l/single-model-for-geology-forecast-challenge) 

The dataset ["geo_submission"](https://www.kaggle.com/datasets/act18l/geo-submission/data) contains  the results from  versions of the above notebooks.



["Emsemble"](https://www.kaggle.com/code/act18l/ensemble-for-geology-forecast-challenge)

---

In the competition, I chose to  **"Single Model(Version 25)"** and **"Ensemble(Version 9)"** as my submission.

You can use "Compare Versions" to check the code changes.

---

**Updated:** Cleaned up the code and added comments.

## Import Library

In [ ]:
import numpy as np
import pandas as pd 
from scipy.special import logsumexp
import os
import random
import numpy as np
import tensorflow as tf
import warnings
 
warnings.filterwarnings("ignore")
# fix seed
# seed = 42
# os.environ['PYTHONHASHSEED'] = str(seed)  # fix python
# random.seed(seed)                       # fix python
# np.random.seed(seed)                    # fix numpy
# tf.random.set_seed(seed)                # fix tensorflow

## Read Data

In [ ]:
%%time
import pandas as pd
import random

#Randomly read 210,000 rows.
filename = "/kaggle/input/notebook2241bf09e2/train_optimized_final.csv"
n_to_sample = 210000

num_lines = 314360

if num_lines <= 0:
    print("Empty.")
else:
    if n_to_sample > num_lines:
        print(f" ({n_to_sample}) > ({num_lines}).Will read all rows.")
        df = pd.read_csv(filename)
    else:
        all_indices = list(range(num_lines))
        sample_indices = sorted(random.sample(all_indices, n_to_sample))
        skip_rows = sorted(random.sample(range(1, num_lines + 1), num_lines - n_to_sample))
        rows_to_keep_0_indexed = sorted(random.sample(range(num_lines), n_to_sample))
        merge_df = pd.read_csv(filename,
                         skiprows=lambda i: i > 0 and i -1 not in rows_to_keep_0_indexed).fillna(0)
    print(f"Read {len(merge_df)} rows.")
    print(merge_df.head())

## Save space

In [ ]:
%%time 
merge_df['geology_id'] = 0 #we don't need geology_id.
merge_df['geology_id'] = merge_df['geology_id'].astype("category")#To save space, we convert them to the category type.

In [ ]:
#Compress the integer and float numerical types.
import pandas as pd
def downcast_all(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    int_cols = df.select_dtypes(include=['int64', 'int32']).columns
    df[int_cols] = df[int_cols].apply(pd.to_numeric, downcast='integer')

    float_cols = df.select_dtypes(include=['float64', 'float32']).columns
    df[float_cols] = df[float_cols].apply(pd.to_numeric, downcast='float')

    return df

print("Before: {:.2f} MB".format(merge_df.memory_usage(deep=True).sum() / 1024**2))
merge_df = downcast_all(merge_df)
print("After : {:.2f} MB".format(merge_df.memory_usage(deep=True).sum() / 1024**2))

## Read and merge

In [ ]:
train=pd.read_csv("/kaggle/input/geology-forecast-challenge-open/data/train.csv").fillna(0)
test=pd.read_csv("/kaggle/input/geology-forecast-challenge-open/data/test.csv").fillna(0)
sub=pd.read_csv('/kaggle/input/geology-forecast-challenge-open/data/sample_submission.csv')
train.shape,test.shape,sub.shape

In [ ]:
train['geology_id'] = 0
test['geology_id'] = 0
train['geology_id'] = train['geology_id'].astype("category")
test['geology_id'] = test['geology_id'].astype("category")
train = downcast_all(train)
test = downcast_all(test)
train.head()

In [ ]:
merge_df = pd.concat([train,merge_df],axis=0)
train = merge_df
train.shape

In [ ]:
FEATURES=[c for c in test.columns if c!='geology_id']
TARGETS=[c for c in sub.columns if c!='geology_id']
solution=train[['geology_id']+TARGETS].copy()
train_sub=train[['geology_id']+TARGETS].copy()
train_sub.head()

## Word2Vec

In [ ]:
VECTOR_SIZE = 0 #we don't use Word2Vec
if VECTOR_SIZE==0:
    train = train.drop('geology_id',axis=1)
    test = test.drop('geology_id',axis=1)
    train.head()
else:
    from gensim.models import Word2Vec

    sentences = [train['geology_id'].tolist(),test['geology_id'].tolist()]  

    model = Word2Vec(sentences, vector_size=VECTOR_SIZE, window=10, min_count=1, sg=1)
    
    train = pd.concat([pd.DataFrame(train['geology_id'].apply(lambda x: model.wv[x]).tolist(), columns=[f'vec_{i}' for i in range(VECTOR_SIZE)]),train],axis=1).drop("geology_id",axis=1)
    test = pd.concat([pd.DataFrame(test['geology_id'].apply(lambda x: model.wv[x]).tolist(), columns=[f'vec_{i}' for i in range(VECTOR_SIZE)]),test],axis=1).drop("geology_id",axis=1)
    
    train.head()

## Add new feature

In [ ]:
NEW_FEATURE=0#we don't add new features
def add_diff_feature(df):
    cols = [col for col in df.columns if col.lstrip('-').isdigit()]
    cols = [col for col in cols if -299 <= int(col) <= 0]
    cols = sorted(cols, key=lambda x: int(x))
    diff_dict = {}
    for i in range(0, len(cols)-1,20):
        col1 = cols[i]
        col2 = cols[i+1]
        new_col_name = f"diff_{col1}_{col2}"
        diff_dict[new_col_name] = df[col1].astype(float) - df[col2].astype(float)
    df_diff = pd.DataFrame(diff_dict)
    df = pd.concat([df_diff,df], axis=1)
    print(df_diff.shape)
    return df
    
# train = add_diff_feature(train)
# test = add_diff_feature(test)

## Score function

In [ ]:
NEGATIVE_PART = -299
LARGEST_CHUNK = 600
TOTAL_REALIZATIONS = 10
INFLATION_SIGMA = 600
num_columns = LARGEST_CHUNK + NEGATIVE_PART - 1 

def compute_sigma(num: int) -> np.ndarray:
    sigma = np.ones(num)
    segments = [
        (1, 61, 1.0406028049510443, -6.430669850650689),
        (61, 245, 0.0, -2.1617411566043896),
        (245, 301, 7.835345062351012, -45.24876794412965)
    ]
    for start, end, slope, offset in segments:
        indices = np.arange(start, end)  
        sigma[indices - 1] = np.exp(np.log(indices) * slope + offset)
    return sigma * INFLATION_SIGMA

cov_inv_diag = 1. / compute_sigma(num_columns)
# ---------------- Simplified code ----------------
def score_simplified(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    solution = solution.drop(columns=[row_id_column_name])
    submission = submission.drop(columns=[row_id_column_name])
    num_rows = solution.shape[0]
    inner_product_matrix = np.empty((num_rows, TOTAL_REALIZATIONS))
    
    cols_list = [
        [str(i+1) if k == 0 else f"r_{k}_pos_{i+1}" for i in range(num_columns)]
        for k in range(TOTAL_REALIZATIONS)
    ]
    solution_arr = np.stack([solution[cols].values for cols in cols_list], axis=1)
    submission_arr = np.stack([submission[cols].values for cols in cols_list], axis=1)

    ##core metric
    # shape=(num_rows, TOTAL_REALIZATIONS, num_columns)
    misfit = solution_arr - submission_arr
    inner_product_matrix = np.sum(cov_inv_diag * misfit  * misfit, axis=2)
    print(logsumexp(inner_product_matrix -np.log(TOTAL_REALIZATIONS), axis=1).shape)
    score_val = -logsumexp(inner_product_matrix, axis=1).mean()+np.log(TOTAL_REALIZATIONS)
    ##core metric
    return score_val

## Training and model

In [ ]:
X = np.array(train.iloc[:,0:300+VECTOR_SIZE+NEW_FEATURE]).astype('float32')
y = np.array(train.iloc[:,300+VECTOR_SIZE+NEW_FEATURE:600+VECTOR_SIZE+NEW_FEATURE]).astype('float32')
X_test = np.array(test.iloc[:,0:300+VECTOR_SIZE+NEW_FEATURE]).astype('float32')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Loss
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Add,Permute,GlobalAveragePooling1D,Input,GlobalMaxPooling1D,SeparableConv1D, Lambda, Dense, Conv1D, MultiHeadAttention, LayerNormalization, Dropout
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from tensorflow.keras.layers import Input, Dense, Lambda, LayerNormalization, MultiHeadAttention, Conv1D, BatchNormalization, GlobalAveragePooling1D

## custom loss for metric
class CustomLoss(Loss):
    def __init__(self, k_matrix):
        super().__init__()
        self.k_matrix = k_matrix

    def call(self, y_true, y_pred):
        diff = y_true - y_pred
        loss = diff * tf.convert_to_tensor(self.k_matrix,dtype=tf.float32) * diff
        return keras.backend.mean(loss)
        
## Model Architecture
def build_model(input_dim, output_dim):
    inputs = Input(shape=(input_dim,))
    col_indices = tf.range(1, 301, dtype=tf.float32)  
    inputs =  col_indices + inputs  
    x = Lambda(lambda t: tf.expand_dims(t, axis=1))(inputs) #For Conv1D
    x = Conv1D(filters=64, kernel_size=7, padding='same', activation='relu')(x) #Need 3d input
    mha = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    mha = LayerNormalization()(mha + x)  
    mha = GlobalAveragePooling1D()(mha)
    ffn = Dense(512, activation="relu")(mha)
    ffn = Dense(64, activation="relu")(ffn)
    ffn = Dense(output_dim, activation="linear")(ffn)
    model = keras.Model(inputs, ffn)  
    #print(model.summary())
    return model

def cross_validate(X, y, X_test, k_matrix, n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    val_losses = []
    y_oof = np.zeros_like(y, dtype=np.float32)
    y_test=[]
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"Training on fold {fold + 1}/{n_folds}")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        #scale to x
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        X_test1 = scaler.transform(X_test)

        #scale to y
        scaler_y = StandardScaler()
        y_train = scaler_y.fit_transform(y_train)
        y_val = scaler_y.transform(y_val)

        model = build_model(input_dim=X.shape[1], output_dim=y.shape[1])
        model.compile(optimizer=Adam(learning_rate=0.0001),
                      loss=CustomLoss(k_matrix))
        history = model.fit(X_train, 
                            y_train,
                            validation_data=(X_val, y_val),
                            epochs=20,
                            batch_size=128,
                            verbose=2)

        val_loss = history.history['val_loss'][-1]
        val_losses.append(val_loss)
        print(f"Fold {fold + 1} validation loss: {val_loss}")
        y_oof[val_idx] = np.squeeze(scaler_y.inverse_transform(model.predict(X_val,verbose=0)))
        y_test.append(np.squeeze(scaler_y.inverse_transform(model.predict(X_test1,verbose=0))))
    avg_val_loss = np.mean(val_losses)
    print(f"Average validation loss across {n_folds} folds: {avg_val_loss}")
    return avg_val_loss,y_oof,y_test

input_dim = 300+VECTOR_SIZE
output_dim = 300

_,y_oof,y_test = cross_validate(X, y, X_test, cov_inv_diag)

## Repeat

As mentioned [here](https://www.kaggle.com/competitions/geology-forecast-challenge-open/discussion/569884#3159115), the best strategy is to copy 300 rows ten times.

In [ ]:
group_size = 300

num_groups = (train_sub.shape[1] + group_size - 1) // group_size-1

for i in range(num_groups):
    start_col = i * group_size+1
    end_col = (i + 1) * group_size+1
    train_sub.iloc[:, start_col:end_col] = y_oof

In [ ]:
print(f"final CV:{score_simplified(solution,train_sub,'geology_id')}")

# submission

In [ ]:
group_size = 300

num_groups = (sub.shape[1] + group_size - 1) // group_size-1

for i in range(num_groups):
    start_col = i * group_size+1
    end_col = (i + 1) * group_size+1
    sub.iloc[:, start_col:end_col] = np.mean(y_test,axis=0)
sub.to_csv("submission.csv",index=None)
sub.head()